## PyMC3 Implementation of Chempy
First create a basic Chempy implementation in Chempy for a single end-time and single star

In [396]:
%pylab inline
import scipy,theano,os,inspect,platform
import pymc3 as pm
import numpy.lib.recfunctions as rcfuncs
import numpy as np
import theano.tensor as tt

## TESTING PARAMETERS
full_checking = False # if True, check assertions in code (will be slower)

# Import useful Chempy functions
os.chdir('/home/oliverphilcox/ChempyMulti/')
from Chempy.solar_abundance import solar_abundances as SolarAbundances
from Chempy.infall import PRIMORDIAL_INFALL as PrimordialInfall

# Define localpath
if platform.system()=='Windows':
    string='\\'
else:
    string='/'

localpath = string.join(os.path.abspath(inspect.getfile(inspect.currentframe())).split(string)[:-1])
localpath += string+'Chempy'+string

if full_checking:
    # Define incomplete gamma function as theano operator
    from theano.compile.ops import as_op
    from scipy.special import gammainc

    @as_op(itypes=[tt.dscalar,tt.dscalar],
        otypes=[tt.dscalar])

    def gammainc_th(shape_par,x):
        return gammainc(shape_par,x)


Populating the interactive namespace from numpy and matplotlib


In [397]:

### MODELLING PARAMETERS

class Parameters(object):
    """This holds various parameters for the PyMC3 Chempy implementation"""

    class general(object):
        # General parameters
        elements_to_trace = ['C', 'Fe', 'H', 'He', 'Mg', 'N', 'Ne', 'O', 'Si']
        gas_reservoir_mass_factor = 1.  # how much more mass does the corona has compared to the integrated SFR
        gas_power = 1. # The Schmidt_exponent (default = 1, i.e. linear)
        sfr_factor_for_cosmic_accretion = 0.000001 # how much more gas should be infalling in the corona compared to the SFR

        # Derived parameters
        n_el = len(elements_to_trace) # number of elements
        # Get indices of elements which are metals and what is H:
        metal_list=[]
        for e,el in enumerate(elements_to_trace):
            if el not in ['H','He']:
                metal_list.append(e)
            if el=='H':
                H_index = e
        metal_list = tt._shared(np.array(metal_list))

    class imf(object):
        # IMF related parameters
        min_mass = 0.1
        max_mass = 100.
        n_mass_steps = 5000 # number of mass steps used

        # Chabrier parameters
        chabrier_A = 0.852464
        chabrier_B = 0.237912
        chabrier_sigma = 0.69
        chabrier_m_c = 0.079 

    class sfr(object):
        # SFR related parameters
        a_parameter = 2 # gamma function shape parameter

    class times(object):
        # Chempy start and end time in Gyr
        start = 0
        end = 13.797617 # Planck 2015 age
        time_steps = 28 # number of time steps in simulation

        star_age = 4.56 # solar age   

        assert (star_age <= 13.0), "Age of the star must be below 13Gyr"
        new_end = end - star_age # time to stop the Chempy run

    class yields(object):
        # Parameters controlling priors on nucleosynthetic yield parameters
        # Format is (intercept, mass coefficient, log_10(Z) coefficient) for each element and data-set
        agb_yield_mean = np.asarray([0.01,0.001,0.001]) # prior mean (in mass fraction)
        agb_yield_std = np.asarray([0.1,0.01,0.01]) # prior standard deviation (in mass fraction)
        sn2_yield_mean = np.asarray([0.01,0.001,0.001]) 
        sn2_yield_std = np.asarray([0.1,0.01,0.01]) 
        sn1a_yield_mean = np.asarray([0.01,0.001]) # no mass dependence here
        sn1a_yield_std = np.asarray([0.1,0.01]) 

        # Also define parameters for remnant mass fraction in same format
        agb_remnant_mean = np.asarray([0.1,0.001,0.001]) # prior mean
        agb_remnant_std = np.asarray([0.2,0.01,0.01]) # prior standard deviation
        sn2_remnant_mean = np.asarray([0.1,0.001,0.001])
        sn2_remnant_std = np.asarray([0.2,0.01,0.01]) 
        sn1a_remnant_mean = np.asarray([-1.37,0.001]) # no mass dependence here
        sn1a_remnant_std = np.asarray([0.2,0.01]) 


    class sn2(object):
        # Define SN2 mass ranges
        min_mass = 8.
        max_mass = 100.

    class agb(object):
        # Define AGB mass ranges
        min_mass = 1.
        max_mass = 8.

    class sn1a(object):
        # Define Maoz (2012) DTD parameters
        tau_8 = 0.04 # this is the SN1a time delay used in TNG
        s_exponent = 1.12

In [460]:

### PRELOADING

# Load parameters
par = Parameters()

class IMF(object):
    # Preload IMF from Chabrier (2003)

    def __init__(self):
        masses = np.linspace(par.imf.min_mass,par.imf.max_mass,par.imf.n_mass_steps)
        masses_low = masses[masses<1.]
        masses_high = masses[masses>1.]

        # Define lower end of IMF (in Numpy first)
        dn_low = par.imf.chabrier_A/masses_low*np.exp(-(np.log10(masses_low/par.imf.chabrier_m_c)**2.)/(2.*par.imf.chabrier_sigma**2.))
        self.dn_low = tt._shared(dn_low) # convert to tensors
        self.masses = tt._shared(masses)
        self.masses_high = tt._shared(masses_high)
        self.masses_low = tt._shared(masses_low)

    def initialize_slope(self,alpha_imf):
        # Compute the IMF given the high-mass slope (needs to be done inside PyMC3 model)
        self.dn_high = par.imf.chabrier_B*self.masses_high**alpha_imf
        self.dn = tt.concatenate([self.dn_low,self.dn_high])*self.masses  # number of stars produced per unit mass
        self.dm = self.dn/self.dn.sum() # mass of stars produced per unit mass
        self.dn = self.dm/self.masses

    def imf_mass_fraction(self,mlow,mup):
        # This determines the mass fraction of the IMF between mlow and mup
        mup = tt.switch(tt.isnan(mup),par.imf.max_mass,mup) # remove any NaN values from upper limit
        cut=tt.and_(self.masses>=mlow,self.masses<mup)
        fraction = self.dm[cut].sum()
        return fraction

    def imf_number_fraction(self,mlow,mup):
        # This determines the number fraction of the IMF between mlow and mup
        mup = tt.switch(tt.isnan(mup),par.imf.max_mass,mup) # remove any NaN values from upper limit
        cut=tt.and_(self.masses>=mlow,self.masses<mup)
        fraction = self.dn[cut].sum()
        return fraction

class SFR(object):
    # Preload star formation rate (here a Gamma function with shape parameter a = 2 by default)
    # We assume SFR starts at t=0 here

    # Define evolution time steps (first in Numpy)
    t = np.linspace(par.times.start,par.times.new_end,par.times.time_steps)
    dt = t[1]-t[0]
    #timespan = par.times.new_end - par.times.start

    # convenience functions
    tmp_t1 = np.power(t,par.sfr.a_parameter-1)
    tmp_t2 = np.exp(-1.*t)
    t = tt._shared(t) # convert to tensors
    tmp_t1 = tt._shared(tmp_t1)
    tmp_t2 = tt._shared(tmp_t2)
        
    def initialize_scale(self,sfr_scale):
        # Load the SFR in full given the SFR scale. This must be run in the PyMC3 model

        # First compute normalized gamma function SFR up to the correct end time
        sfr = self.tmp_t1*tt.power(self.tmp_t2,1./sfr_scale) # unnormalized gamma function
        self.sfr = sfr/(sfr.sum())

        if full_checking:
            # Check modeling assumptions
            tt.opt.Assert(self.sfr[0]==0.)

            # Now compute the total mass formed in this simulation (normalized to unit mass formed in universe age)
            total_mass = gammainc_th(theano.shared(1.*par.sfr.a_parameter),par.times.new_end/sfr_scale)
            total_mass_univ = gammainc_th(theano.shared(1.*par.sfr.a_parameter),par.times.end/sfr_scale) # total mass formed in universe age
            total_mass /= total_mass_univ

            # Check we form enough stars
            mean_sfr = total_mass_univ/par.times.end
            sfr_at_end = self.sfr[-1]
            tt.opt.Assert(sfr_at_end>0.05*mean_sfr)

class Infall(SFR):
    # Preload infall mass over time class (matched to SFR class)
    # This inherits time vector from SFR class
    # We assume SFR-related infall here
    def __init__(self):
        self.infall = tt.zeros_like(self.t)

print("Can remove lifetime function...")

# def lifetime_function(m,Z):
#     """
#     Lifetime function from Portinari+ 1998, as used in IllustrisTNG.
#     Inputs are mass in Msun and metallicity fraction.        
#     This returns lifetime of start in Gyrs.
#     """

#     lM = np.log10(m)
#     if Z<=0.:
#         lZ=-5
#     else:
#         lZ = np.log10(Z) # to avoid zero errors
#     params = np.array([10.0153615 , -3.91089893,  0.99947209, -0.03601771, -0.31139679,0.09109059, -0.03218365, -0.01323112])
#     tmp = (params[0]+lM*params[1]+np.power(lM,2.)*params[2]+lZ*params[3]+lM*lZ*params[4]+lM*lM*lZ*params[5]+lZ*lZ*params[6]+lZ*lZ*lM*params[7])
#     return np.divide(np.power(10,tmp),1e9)    

def inverse_lifetime_function(lifetime,Z):
    """Inverse of the lifetime function from Portinari+ 1998, as used in IllustrisTNG.
    For this parametrization, the lifetime can be computed analytically.
    Inputs are age in Gyr and metallicity fraction.
    This returns the mass in Msun"""
    lZ = tt.log10(tt.max([Z,1e-5])) # to avoid zero errors
    lZ2 = lZ*lZ
    params = np.array([10.0153615 , -3.91089893,  0.99947209, -0.03601771, -0.31139679,0.09109059, -0.03218365, -0.01323112])
    #params = tt._shared(params)
    alpha = params[0]+lZ*params[3]+lZ2*params[6]
    beta = params[1]+lZ*params[4]+lZ2*params[7]
    gamma = params[2]+lZ*params[5]
    
    t = 9+tt.log10(lifetime)
    disc = beta**2.-4.*(alpha-t)*gamma
    
    def output(disc):
        # Compute logM from quadratic;
        lM = (-beta-tt.sqrt(disc))/(2.*gamma)
        return tt.power(10.,lM)
    
    # Ensure we return max IMF value for too-small-time errors
    return tt.switch(disc>0,output(disc),par.imf.max_mass)

def sn1a_time_delay(time_array,dt,sn1a_normalization,tau_8=par.sn1a.tau_8,s_exponent=par.sn1a.s_exponent):
    """ Compute the SN1a delay time distribution (DTD), assuming the form given in Maoz 2012.
    tau_8 is the delay time parameter.
    
    This is computed for an array of input times, separated by dt
    
    Unlike in Chempy we normalize by integrating the DTD instead of summing so it is independent of the time-step 
    """
    
    # Create DTD and zero any times with time<tau_8
    DTD = tt.switch(time_array>tau_8,tt.power(time_array/tau_8,-1.*s_exponent)*(s_exponent-1.)/tau_8,0)

    norm = (1.-tt.power((par.times.end/tau_8),1.-s_exponent))/dt
    
    # Normalize correctly
    return DTD/norm*sn1a_normalization
    

def mass_fractions_to_abundances(mass_fractions):
    """ Convert an array of mass fractions to [X/Fe] abundances normed to Asplund 2009 solar values."""
    
    output = mass_fractions/solar_abundances.cut_masses # convert to number fractions
    output/=output[par.general.H_index] # normalize by H 
    
    tt.opt.Assert(output>=0.) # check positive mass fractions
    
    X_H_abundances = tt.log10(output)+12.-solar_abundances.cut_abundances
    
    # Convert to X/Fe abundances
    X_Fe_abundances = X_H_abundances - X_H_abundances[par.general.Fe_index]
    # Also use [Fe/H] abundance
    X_Fe_abundances = tt.set_subtensor(X_Fe_abundances[par.general.Fe_index], X_H_abundances[par.general.Fe_index])
    
    return X_Fe_abundances

Can remove lifetime function...


In [461]:

class AbundanceMatrix(object):
    # Preload important quantities for holding the chemical evolution properties of Chempy
    # self.ism contains the ISM evolution, self.gas_reservoir contains the corona evolution 
    # and self.sn2/sn1a/agb_feedback has individual process enrichments

    class ISM(object):
        """Class to hold useful quantities for the ISM at each timestep"""
        def __init__(self,time_steps):
            print("Check we use all these and they have a purpose")
            self.mass_fractions = tt.zeros((time_steps,par.general.n_el))
            self.gas = tt.zeros(time_steps)
            self.Z = tt.zeros(time_steps)
            self.infall = tt.zeros(time_steps)
            self.feedback = tt.zeros(time_steps)
            self.mass_in_remnants = tt.zeros(time_steps)
            self.stars = tt.zeros(time_steps)
            self.sn1a_events = tt.zeros(time_steps)
            self.sn2_events = tt.zeros(time_steps)
            self.agb_events = tt.zeros(time_steps)

    class Corona(object):
        """Class to hold useful quantities for the gas reservoir (corona) at each timestep"""
        def __init__(self,time_steps):
            self.mass_fractions = tt.zeros((time_steps,par.general.n_el))
            self.gas = tt.zeros(time_steps)
            self.Z = tt.zeros(time_steps)
            self.feedback = tt.zeros(time_steps)

    class AllFeedback(object):
        """Class to hold useful quantities giving amount of feedback used at each timestep from each timestep"""
        def __init__(self,time_steps):
            self.yields = tt.zeros((time_steps,time_steps,par.general.n_el))
            self.sn2_events = tt.zeros((time_steps,time_steps))
            self.agb_events = tt.zeros((time_steps,time_steps))
            self.sn1a_events = tt.zeros((time_steps,time_steps))
            self.remnants = tt.zeros((time_steps,time_steps))
            
    def __init__(self,SFR,Infall,PrimordialInfall):
        # Copy useful variables
        ##TODO: do we need all these?
        self.t = SFR.t
        self.dt = SFR.dt
        self.infall = Infall.infall
        self.elements = par.general.elements_to_trace
        self.infall_symbols = PrimordialInfall.symbols
        self.infall_fractions = tt._shared(PrimordialInfall.fractions)
        self.sfr_factor_for_cosmic_accretion = par.general.sfr_factor_for_cosmic_accretion

        ##TODO: DO WE NEED THESE?
        #self.cosmic_accretion_elements = PrimordialInfall.symbols
        #self.cosmic_accretion_element_fractions = PrimordialInfall.fractions

        # Define classes
        self.ism = self.ISM(par.times.time_steps)
        self.corona = self.Corona(par.times.time_steps)
        self.all_feedback = self.AllFeedback(par.times.time_steps)

        # Add initial gas composition
        # (since we have no initial gas and SFR=0 at t=0 this is independent of free parameters)
        starting_gas = par.general.gas_reservoir_mass_factor
        self.corona.gas=tt.set_subtensor(self.corona.gas[0],starting_gas)
        self.corona.mass_fractions=tt.set_subtensor(self.corona.mass_fractions[0,:],starting_gas*self.infall_fractions)
        # Compute initial metallicity (zero for ISM)
        self.corona.Z = tt.set_subtensor(self.corona.Z[0],self.corona.mass_fractions[0,par.general.metal_list].sum()/starting_gas)

        # Initialize ISM gas to negligible mass with correct mass ratios
        self.ism.gas = tt.set_subtensor(self.ism.gas[0],starting_gas*1e-20)
        self.ism.mass_fractions = tt.set_subtensor(self.ism.mass_fractions[0,:],self.corona.mass_fractions[0,:]*1e-20)
        self.ism.Z = tt.set_subtensor(self.ism.Z[0],self.ism.mass_fractions[0,par.general.metal_list].sum()/self.ism.gas[0])
        

In [462]:
class SSP(object):
    """Class to hold an SSP object giving its enrichment over time""" 
    def __init__(self,z,times,fractions_in_gas,imf):
        """z = metallicity in mass fraction (i.e. Z).
        times = times over which to compute the SSP enrichment
        fractions_in_gas = initial SSP mass fractions (for gross feedback)
        """
        self.fractions_in_gas = fractions_in_gas
        self.z = z
        self.t = times
        self.dt = times[1]-times[0]
        self.imf = imf
        
        self.logZ = tt.log10(tt.max([self.z,1e-5])) # to avoid zero errors
        self.time_steps_trunc = tt.shape(self.t)

        # Compute inverse IMF function
        self.inverse_imf = inverse_lifetime_function(self.t,self.z)

        # Create class to hold enrichment table
        class EnrichmentTable(object):
            # SSP enrichment table
            def __init__(self,n_steps):
                # n_steps is length of table
                
                self.mass_in_ms_stars=tt.zeros(n_steps)
                self.mass_of_ms_stars_dying=tt.zeros(n_steps)
                
                print("Can we define this as one-step shorter?")
                self.sn1a_events = tt.zeros(n_steps)
                self.sn2_events = tt.zeros(n_steps)
                self.agb_events = tt.zeros(n_steps)
                self.yields = tt.zeros((n_steps[0],par.general.n_el))
                self.remnants = tt.zeros(n_steps)
                
                self.hydrogen_mass_accreted_onto_white_dwarfs = tt.zeros(n_steps)
                
                self.tmp = n_steps
                
        self.table=EnrichmentTable(self.time_steps_trunc)

        # Compute mass of MS stars in each time-step and add to table
        scan_results,_ = theano.scan(lambda this_mlow,this_mup: self.imf.imf_mass_fraction(this_mlow,this_mup),
                                     sequences=[self.inverse_imf[1:],self.inverse_imf[:-1]])
        
        self.table.mass_of_ms_stars_dying = tt.set_subtensor(self.table.mass_of_ms_stars_dying[1:],scan_results)
        self.table.mass_in_ms_stars = 1.-tt.cumsum(self.table.mass_of_ms_stars_dying)

    def sn2_feedback(self):
        """ Compute the SN2 feedback table from the SSP initialized above.
        """
        
        # First count the number of SN2 events
        print("Is this even used?? [good as a scan check though]")
        def count_sn2(inv_imf_low,inv_imf_high):
            # Count number of SN2 in this mass range
            # We pass in an inverse IMF interval [inv_imf_low,inv_imf_high]
            # which is strictly decreasing
            
            mlow = tt.max([inv_imf_high,par.sn2.min_mass])
            mup = tt.min([inv_imf_low,par.sn2.max_mass])
            
            return tt.switch(tt.and_(mlow>mup,inv_imf_high<par.sn2.min_mass),0.,imf.imf_number_fraction(mlow,mup))
            
        scan_results,_ = theano.scan(count_sn2,
                                     sequences=[self.inverse_imf[:-1],self.inverse_imf[1:]])
        self.table.sn2_events = tt.set_subtensor(self.table.sn2_events[1:],scan_results)

        def feedback_per_time_step(inv_imf_low,inv_imf_high):
            # Compute the SN2 feedback for each time-step.
            # This wil be looped in a theano scan function

            # Need to sum over masses in correct region allowed by IMF
            lower_cut = tt.max([inv_imf_high,par.sn2.min_mass])
            upper_cut = tt.min([inv_imf_low,par.sn2.max_mass])

            # Here looking at all masses in correct range
            cut = tt.and_(imf.masses<upper_cut,imf.masses>=lower_cut)

            print("This assumes a linear weight function in mass")
            sum_weights = imf.dm[cut].sum()
            av_mass = (imf.masses[cut]*imf.dm[cut]).sum()/sum_weights
            sn2_yields = sn2_yield(av_mass,self.logZ)*sum_weights

            sn2_remnants = sn2_remnant(av_mass,self.logZ)*sum_weights
            
            # Now add unprocessed mass to gross yields
            sn2_yields += (sum_weights-sn2_remnants)*self.fractions_in_gas

            print("NB: all tables are gross tables here unlike Chempy")

            return tt.switch(inv_imf_low<par.sn2.min_mass,tt.zeros_like(sn2_yields),sn2_yields),tt.switch(inv_imf_low<par.sn2.min_mass,0.,sn2_remnants)

        scan_results,_ = theano.scan(feedback_per_time_step,sequences=[self.inverse_imf[:-1],self.inverse_imf[1:]])

        self.table.yields = tt.set_subtensor(self.table.yields[1:],self.table.yields[1:]+scan_results[0])
        self.table.remnants = tt.set_subtensor(self.table.remnants[1:],self.table.remnants[1:]+scan_results[1])
        
    def agb_feedback(self):
        """ Compute the AGB feedback table from the SSP initialized above.
        """
        # First count the number of AGB events
        print("Is this even used?? [good as a scan check though]")
        def count_agb(inv_imf_low,inv_imf_high):
            # Count number of AGB in this mass range
            # We pass in an inverse IMF interval [inv_imf_low,inv_imf_high]
            # which is strictly decreasing
            
            mlow = tt.max([inv_imf_high,par.agb.min_mass])
            mup = tt.min([inv_imf_low,par.agb.max_mass])
            
            return tt.switch(tt.and_(mlow>mup,inv_imf_high<par.agb.min_mass),0.,imf.imf_number_fraction(mlow,mup))
            
        scan_results,_ = theano.scan(count_agb,
                                     sequences=[self.inverse_imf[:-1],self.inverse_imf[1:]])
        self.table.agb_events = tt.set_subtensor(self.table.agb_events[1:],scan_results)
        
        def feedback_per_time_step(inv_imf_low,inv_imf_high):
            # Compute the AGB feedback for each time-step.
            # This wil be looped in a theano scan function

            # Need to sum over masses in correct region allowed by IMF
            lower_cut = tt.max([inv_imf_high,par.agb.min_mass])
            upper_cut = tt.min([inv_imf_low,par.agb.max_mass])

            # Here looking at all masses in correct range
            cut = tt.and_(imf.masses<upper_cut,imf.masses>=lower_cut)

            sum_weights = imf.dm[cut].sum()
            av_mass = (imf.masses[cut]*imf.dm[cut]).sum()/sum_weights
            agb_yields = agb_yield(av_mass,self.logZ)*sum_weights

            agb_remnants = agb_remnant(av_mass,self.logZ)*sum_weights
            
            # Now add unprocessed mass to gross yields
            agb_yields += (sum_weights-agb_remnants)*self.fractions_in_gas

            print("NB: agb tables are treated weirdly in Chempy - possibly missing first/last step, so may be different")

            return tt.switch(inv_imf_low<par.agb.min_mass,tt.zeros_like(agb_yields),agb_yields),tt.switch(inv_imf_low<par.agb.min_mass,0.,agb_remnants)

        scan_results,_ = theano.scan(feedback_per_time_step,sequences=[self.inverse_imf[:-1],self.inverse_imf[1:]])

        # Now add to global yield tables
        self.table.yields = tt.set_subtensor(self.table.yields[1:],self.table.yields[1:]+scan_results[0])
        self.table.remnants = tt.set_subtensor(self.table.remnants[1:],self.table.remnants[1:]+scan_results[1])
        
    def sn1a_feedback(self,sn1a_normalization):
        """ Compute the SNIa feedback table from the SSP initialized above.
        
            sn1a_normalization is the DTD normalization (number of SN1a per 1Msun per 13.8 Gyr)
        """
        
        # Compute the number of supernovae in each time-step (from the DTD) 
        sn1a_feedback_number = sn1a_time_delay(self.t,self.dt,sn1a_normalization)
        
        # Set feedback mass parameters
        print("Should this be fit or set from data? (well constrained)")
        mean_mass_of_feedback = -sn1a_remnant(self.logZ) # This mass is turned into the explosion

        print("These seem arbitrarily set... Can we get a better parametrization?")
        mean_mass = 2.156 #mass_of_stars_in_mass_range_for_remnant/number_of_stars_in_mass_range_for_remnant
        mean_remnant_mass = mean_mass*0.3
        mean_accretion_mass = mean_mass_of_feedback - mean_remnant_mass # from Hydrogen feedback
        print("Only mean_remnant_mass is unset here - set as free parameter?")
        
        # Now add quantities to global yield tables
        self.table.remnants -= sn1a_feedback_number * mean_remnant_mass # how much mass will be turned from remnants into feedback (single degenerate scenario)
        print("Do we need hydrogen_mass_accreted_onto_white_dwarfs traced?")
        self.table.hydrogen_mass_accreted_onto_white_dwarfs = sn1a_feedback_number * mean_accretion_mass
        self.table.yields = tt.set_subtensor(self.table.yields[:,par.general.H_index],
                                             self.table.yields[:,par.general.H_index]-sn1a_feedback_number * mean_accretion_mass)
        print("Is number of sn1a_events used?")
        self.table.sn1a_events = sn1a_feedback_number
        
        # Add element yields to each time-step
        tmp_yields = sn1a_yield(self.logZ)*mean_mass_of_feedback
        scan_result,_=theano.scan(lambda feedback_number: tmp_yields*feedback_number,sequences=sn1a_feedback_number)
        self.table.yields += scan_result
        

In [463]:
        
## MAIN CODE

# Pre-load modules
imf = IMF()
sfr = SFR()

print("Is infall class used?")
infall = Infall()

# Load solar abundances from Chempy
solar_abundances = SolarAbundances()
solar_abundances.Asplund09()

# Select correct ranges of solar abundances for later
element_masses = [] # to hold mass of each element
solar_abundances_cut = [] # to hold photospheric abundances of each element
for item in par.general.elements_to_trace:
    element_masses.append(solar_table['Mass'][np.where(solar_table['Symbol']==item)][0])
    solar_abundances_cut.append(solar_table['photospheric'][np.where(solar_table['Symbol']==item)][0])
solar_abundances.cut_masses = tt._shared(np.asarray(element_masses))
solar_abundances.cut_abundances = tt._shared(np.asarray(solar_abundances_cut))

primordial_infall = PrimordialInfall(par.general.elements_to_trace,solar_abundances.table)
primordial_infall.primordial()

Is infall class used?


In [464]:
full_model = pm.Model()

with full_model:

    ## PARAMETERS TO INFER
    class InferenceParameters(object):
        # Main parameters
        alpha_imf = pm.Normal('imf-slope',mu=-2.3,sd=0.3) # high-mass IMF slope
        log10_N_1a = pm.Normal('log10-N-1a',mu=-2.89,sd=0.3) # SN-1a normalization

        log10_sfe = pm.Normal('log10-sfe',mu=-0.3,sd=0.1) # the SFE for a linear Kennicut-Schmidt law
        log10_sfr_scale = pm.Normal('log10-sfr-scale',mu=0.55,sd=0.1) 
        x_out = pm.Normal('x-out',mu=0.5,sd=0.1) # fractional enrichment goes into the corona

        # Derived parameters
        N_1a = tt.pow(10.,log10_N_1a)
        sfr_scale = tt.pow(10.,log10_sfr_scale)
        sfe = tt.pow(10.,log10_sfe)

        ## Yield parameters
        print("How do we fix yields to be less than unity? All elements should sum to 1...")
        # Net yield parameters
        sn1a_par = pm.Normal('sn1a-yields',mu=par.yields.sn1a_yield_mean,sd=par.yields.sn1a_yield_std,shape=(par.general.n_el,2))
        sn2_par = pm.Normal('sn2-yields',mu=par.yields.sn2_yield_mean,sd=par.yields.sn2_yield_std,shape=(par.general.n_el,3))
        agb_par = pm.Normal('agb-yields',mu=par.yields.agb_yield_mean,sd=par.yields.agb_yield_std,shape=(par.general.n_el,3))

        # Remnant mass fraction parameters
        sn1a_remnant = pm.Normal('sn1a-remnant',mu=par.yields.sn1a_remnant_mean,sd=par.yields.sn1a_remnant_std,shape=2)
        sn2_remnant = pm.Normal('sn2-remnant',mu=par.yields.sn2_remnant_mean,sd=par.yields.sn2_remnant_std,shape=3)
        agb_remnant = pm.Normal('agb-remnant',mu=par.yields.agb_remnant_mean,sd=par.yields.agb_remnant_std,shape=3)

    inference = InferenceParameters()
        
    cube=AbundanceMatrix(sfr,infall,primordial_infall)


How do we fix yields to be less than unity? All elements should sum to 1...
Check we use all these and they have a purpose


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


In [465]:
with full_model:
    ## Load IMF (Chabrier 2003)
    imf.initialize_slope(inference.alpha_imf)    
    
    ## SFR (Gamma function)
    sfr.initialize_scale(inference.sfr_scale)
    
    ## AbundanceMatrix class
    cube.sfr = sfr.sfr
    cube.star_formation_efficiency = inference.sfe*cube.dt

    ## YIELDS
    # Define functions for mass fraction, remnant mass and unprocessed mass in winds
    print("NB: We use logZ=-5 for Z=0")
    print("NB: No dependence on mass for SN1a yields")
    def sn1a_yield(logZ):
        return inference.sn1a_par[:,0]+inference.sn1a_par[:,1]*logZ
    def sn2_yield(mass,logZ):
        return inference.sn2_par[:,0]+inference.sn2_par[:,1]*mass+inference.sn2_par[:,2]*logZ
    def agb_yield(mass,logZ):
        return inference.agb_par[:,0]+inference.agb_par[:,1]*mass+inference.agb_par[:,2]*logZ
    def sn1a_remnant(logZ):
        return inference.sn1a_remnant[0]+inference.sn1a_remnant[1]*logZ
    def sn2_remnant(mass,logZ):
        return inference.sn2_remnant[0]+inference.sn2_remnant[1]*mass+inference.sn2_remnant[2]*logZ
    def agb_remnant(mass,logZ):
        return inference.agb_remnant[0]+inference.agb_remnant[1]*mass+inference.agb_remnant[2]*logZ


NB: We use logZ=-5 for Z=0
NB: No dependence on mass for SN1a yields


In [467]:
with full_model:

    def advance_simulation(time_index):
        """Function to step through time in the simulation. 
        This must be iterated over by theano's scan function (i.e. a for loop) 

        time_index is the current time-step
        """

        # First load element fractions from the time-step before
        print("Which time step should these be for?")
        element_fractions = cube.ism.mass_fractions[tt.max([time_index-1,0]),:]/cube.ism.gas[tt.max([time_index-1,0])]
        
        if full_checking:
            # Check for unphysical negative element fractions
            for i in range(par.general.n_el):
                tt.opt.Assert(element_fractions[i]>0)
        
        # Store metallicity of this time-step
        metallicity = cube.ism.Z[time_index]
        
        # Compute time steps from now until the end of the simulation
        remaining_time_steps = sfr.t[:par.times.time_steps-time_index].copy()
        
        ## Compute SSP feedback from each process
        ssp = SSP(cube.ism.Z[time_index],remaining_time_steps,element_fractions,imf)
        ssp.sn2_feedback()
        ssp.agb_feedback()
        ssp.sn1a_feedback(inference.N_1a)

        ## Compute required quantities for the next time step
        
        # First fill up all_feedback table from SSP
        cube.all_feedback.yields = tt.set_subtensor(cube.all_feedback.yields[time_index,time_index:,:],ssp.table.yields)
        cube.all_feedback.sn2_events = tt.set_subtensor(cube.all_feedback.sn2_events[time_index,time_index:],ssp.table.sn2_events)
        cube.all_feedback.agb_events = tt.set_subtensor(cube.all_feedback.agb_events[time_index,time_index:],ssp.table.agb_events)
        cube.all_feedback.sn1a_events = tt.set_subtensor(cube.all_feedback.sn1a_events[time_index,time_index:],ssp.table.sn1a_events)
        cube.all_feedback.remnants = tt.set_subtensor(cube.all_feedback.remnants[time_index,time_index:],ssp.table.remnants)
        
        ### UPDATE CORONA AND ISM QUANTITIES
        ## Update via FEEDBACK
        # First update elements
        tmp = cube.all_feedback.yields[:time_index+1,time_index+1,:]*cube.sfr[:time_index+1].reshape((-1,1))
        feedback_mass = tmp.sum(axis=0)
        total_feedback_mass = feedback_mass.sum()
        cube.ism.mass_fractions = tt.set_subtensor(cube.ism.mass_fractions[time_index+1,:],
                                           cube.ism.mass_fractions[time_index,:] + (1. - inference.x_out) * feedback_mass)
        
        # Update counters, remnants and star masses
        tmp = cube.all_feedback.sn2_events[:time_index+1,time_index+1]*cube.sfr[:time_index+1]
        cube.ism.sn2_events = tt.set_subtensor(cube.ism.sn2_events[time_index+1],
                                           cube.ism.sn2_events[time_index] + (1. - inference.x_out) * tmp.sum(axis=0))
        tmp = cube.all_feedback.agb_events[:time_index+1,time_index+1]*cube.sfr[:time_index+1]
        cube.ism.agb_events = tt.set_subtensor(cube.ism.agb_events[time_index+1],
                                           cube.ism.agb_events[time_index] + (1. - inference.x_out) * tmp.sum(axis=0))
        tmp = cube.all_feedback.sn1a_events[:time_index+1,time_index+1]*cube.sfr[:time_index+1]
        cube.ism.sn1a_events = tt.set_subtensor(cube.ism.sn1a_events[time_index+1],
                                           cube.ism.sn1a_events[time_index] + (1. - inference.x_out) * tmp.sum(axis=0))
        tmp = cube.all_feedback.remnants[:time_index+1,time_index+1]*cube.sfr[:time_index+1]
        cube.ism.mass_in_remnants = tt.set_subtensor(cube.ism.mass_in_remnants[time_index+1],
                                           cube.ism.mass_in_remnants[time_index] + (1. - inference.x_out) * tmp.sum(axis=0))
        cube.ism.stars = tt.set_subtensor(cube.ism.stars[time_index+1], cube.ism.stars[time_index] - total_feedback_mass) 
        print("Why do we not multiply by 1-x_out in the line below??")
        cube.ism.feedback = tt.set_subtensor(cube.ism.feedback[time_index+1], total_feedback_mass)
        
        # Also add COSMIC INFLOW to corona
        cosmic_inflow = cube.sfr[time_index+1] * par.general.sfr_factor_for_cosmic_accretion
        cube.corona.gas = tt.set_subtensor(cube.corona.gas[time_index+1],
                                           cube.corona.gas[time_index]+cosmic_inflow
                                           +inference.x_out*total_feedback_mass)
        cube.corona.mass_fractions = tt.set_subtensor(cube.corona.mass_fractions[time_index+1,:],
                                                     cube.corona.mass_fractions[time_index,:]+
                                                     cube.infall_fractions*cosmic_inflow+
                                                     inference.x_out*feedback_mass)
        cube.corona.feedback = tt.set_subtensor(cube.corona.feedback[time_index+1],total_feedback_mass*inference.x_out)
        
        ## Compute INFALL (SFR-related) via SCHMIDT LAW
        print("Precompute this sfr[time_index+1] for efficiency?")
        gas_needed = tt.power(cube.sfr[time_index+1]/cube.star_formation_efficiency,1./par.general.gas_power)
        gas_there = cube.ism.mass_fractions[time_index+1,:].sum()
        infall_needed = tt.max([0.,(gas_needed - gas_there)*1.00000001]) # to avoid less gas being requested than needed due to rounding errors
        infall_needed = tt.switch(infall_needed+gas_there<=cube.sfr[time_index+1],
                                  (cube.sfr[time_index+1]-gas_there)*1.01,infall_needed)
        infall_needed = tt.min([cube.corona.gas[time_index+1],infall_needed])
        cube.infall = tt.set_subtensor(cube.infall[time_index+1],infall_needed)
        cube.ism.infall = tt.set_subtensor(cube.ism.infall[time_index+1],infall_needed)
        
        # Move infall gas from corona to ISM
        print("Combine these with the above? (careful of ordering)")
        cube.ism.mass_fractions = tt.set_subtensor(cube.ism.mass_fractions[time_index+1,:],
                                                  cube.ism.mass_fractions[time_index+1,:]+
                                                  cube.infall[time_index+1]*cube.corona.mass_fractions[time_index+1,:]/cube.corona.gas[time_index+1])
        cube.corona.mass_fractions = tt.set_subtensor(cube.corona.mass_fractions[time_index+1,:],
                                                     cube.corona.mass_fractions[time_index+1,:]-
                                                     cube.infall[time_index+1]*cube.corona.mass_fractions[time_index+1,:]/cube.corona.gas[time_index+1])
        cube.corona.gas = tt.set_subtensor(cube.corona.gas[time_index+1],cube.corona.gas[time_index+1]-cube.infall[time_index+1])

        ## Create STARS by subtracting SFR
        cube.ism.gas = tt.set_subtensor(cube.ism.gas[time_index+1],cube.ism.mass_fractions[time_index+1].sum())
        
        tt.opt.Assert(cube.ism.gas[time_index]>=cube.sfr[time_index+1])
        print("Reorder to combine with above if possible? (careful of ordering)")
        cube.ism.mass_fractions = tt.set_subtensor(cube.ism.mass_fractions[time_index+1,:],
                                                   cube.ism.mass_fractions[time_index+1,:]*(1.-cube.sfr[time_index+1]/cube.ism.gas[time_index+1]))
        cube.ism.gas = tt.set_subtensor(cube.ism.gas[time_index+1],
                                       cube.ism.gas[time_index+1]-cube.sfr[time_index+1])
        cube.ism.stars = tt.set_subtensor(cube.ism.stars[time_index+1],
                                          cube.ism.stars[time_index+1]+cube.sfr[time_index+1])
        
        ## Determine METALLICITY fractions
        cube.ism.Z = tt.set_subtensor(cube.ism.Z[time_index+1],
                                      cube.ism.mass_fractions[time_index+1,par.general.metal_list].sum()/cube.ism.gas[time_index+1])
        cube.corona.Z = tt.set_subtensor(cube.corona.Z[time_index+1],
                                        tt.switch(cube.corona.gas[time_index+1]>0,
                                                  cube.corona.mass_fractions[time_index+1,par.general.metal_list].sum()/cube.corona.gas[time_index+1],
                                                  np.inf))
        
        # Check we have positive values everywhere
        print("How do we deal with breaking of assumptions? Somehow need to return inf etc.")
        tt.opt.Assert(cube.ism.mass_fractions[time_index+1,:]>=0.)
        tt.opt.Assert(cube.ism.gas[time_index+1]>=0.)
        tt.opt.Assert(cube.corona.gas[time_index+1]>=0.)
        
        print("Need to properly return all tensor objects to pass to next iteration")
        print("May be better just to continually update a single remnant mass / yields etc. rather than saving history")
        print("Change return types!!")
        return cube.ism.mass_fractions[time_index+1,:]
        
    # Run the iterator in theano using cube as initial value which is continually updated
    print("Is it best to iterate like this? Or should we pass in Z and elements + relevant cube bits + iterate?")
    scan_results,scan_updates=theano.scan(advance_simulation,sequences=np.arange(0,par.times.time_steps-1))

    # Compute abundances from mass_fractions
    print("Update outputs with new scan_results")
    X_Fe_abundances = mass_fractions_to_abundances(scan_results[-1])
    
    print("Add some infinities to outputs etc.?")
    
    print("Now compare to observational data!")

Is it best to iterate like this? Or should we pass in Z and elements + relevant cube bits + iterate?
Which time step should these be for?
Can we define this as one-step shorter?
Is this even used?? [good as a scan check though]


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


This assumes a linear weight function in mass
NB: all tables are gross tables here unlike Chempy


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


Is this even used?? [good as a scan check though]
NB: agb tables are treated weirdly in Chempy - possibly missing first/last step, so may be different
Should this be fit or set from data? (well constrained)
These seem arbitrarily set... Can we get a better parametrization?
Only mean_remnant_mass is unset here - set as free parameter?
Do we need hydrogen_mass_accreted_onto_white_dwarfs traced?
Is number of sn1a_events used?


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


Why do we not multiply by 1-x_out in the line below??
Precompute this sfr[time_index+1] for efficiency?
Combine these with the above? (careful of ordering)
Reorder to combine with above if possible? (careful of ordering)
How do we deal with breaking of assumptions? Somehow need to return inf etc.
Need to properly return all tensor objects to pass to next iteration
May be better just to continually update a single remnant mass / yields etc. rather than saving history
Change return types!!
Update outputs with new scan_results


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


AttributeError: type object 'general' has no attribute 'Fe_index'

In [ ]:
X_H_abundances.tag.test_value

In [409]:
mass_fractions = primordial_infall.fractions+0.01
mass_fractions/=np.sum(mass_fractions)

In [451]:
mass_fractions_to_abundances(tt._shared(mass_fractions)).tag.test_value

array([ 0.60737564,  0.86995966,  0.        , -0.01745844,  1.13124833,
        1.14060813,  0.88202925,  0.22284026,  1.1584618 ])

In [244]:
#all_t = np.array([ 0.,0.51102285,1.0220457,1.53306856,2.04409141,2.55511426,
#3.06613711,3.57715996,4.08818281,4.59920567,5.11022852,5.62125137,
#               6.13227422,  6.64329707,  7.15431993,  7.66534278 , 8.17636563,  8.68738848,
#9.19841133,9.70943419, 10.22045704])
all_t = np.arange(0.,10.,0.5)
ssp = SSP(0.00,all_t,tt._shared(np.asarray([0.0, 0.0, 0.76, 0.23999999999999996, 0.0, 0.0, 0.0, 0.0, 0.0])),imf)
#ssp.sn2_feedback()
#print(ssp.table.yields.tag.test_value[:3,:2])
#ssp.agb_feedback()
#print(ssp.table.yields.tag.test_value[:3,:2])
ssp.sn1a_feedback(0.0014)

Can we define this as one-step shorter?
Should this be fit or set from data? (well constrained)
These seem arbitrarily set... Can we get a better parametrization?
Only mean_remnant_mass is unset here - set as free parameter?
Do we need hydrogen_mass_accreted_onto_white_dwarfs traced?


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


Is number of sn1a_events used?


### Theano Scan Testing

In [268]:
theano.config.compute_test_value ='ignore'

def updater(sequence_variable,last_result_1,last_result_2,non_sequence_1,non_sequence_2):
    new_result_1 = tt.set_subtensor(last_result_1[sequence_variable],non_sequence_1)
    new_result_2 = tt.set_subtensor(last_result_2[sequence_variable],non_sequence_2)
    
    return new_result_1, new_result_2

initial_value = tt._shared(np.asarray([5.,3.,4.,5.]))

result,updates = theano.scan(updater,sequences=tt.arange(4),non_sequences=[4.,5.],outputs_info = [initial_value,initial_value])

## Once running:

- Profile code
- Use GPUs
- Check graphs

e.g. see http://www.marekrei.com/blog/theano-tutorial/

## NB:
- Unprocessed mass fraction in winds is 1 - Remnant Mass Fraction here (ignoring small correction terms)
- We assume Maoz time delay form here for SN1a with time-delay of 40 Myr as in TNG
- ISM is Chabrier 2003 as in TNG
- Even if we don't use major elements in the analysis they **should** be included in Chempy since they affect metallicites